## What is poisson ratio for vertex model?

Here is the model:

$$ E = k_A \left(A-A_r\right)^2 + k_P \left(P - P_r\right)^2 $$

### What does the cell look like?
![title](2024_11_22_analytic_deform_img/diagram-20241122.png)



### Finding the area and the perimeter:
$$ A = 6 \cdot A_{triangle} = 6 \cdot \frac{1}{2} \left[\alpha l_i \cdot \beta l_i \frac{\sqrt{3}}{2}\right] $$
$$ A = \frac{3\sqrt{3}}{2}\alpha  \beta l_i^2  $$

$$ P = 2 \cdot \left[\alpha l_i\right] + 4 \left[\frac{l_i}{2} \sqrt{\alpha^2 + 3 \beta^2} \right] $$
$$ P = 2 \alpha l_i + 2 l_i \sqrt{\alpha^2 + 3 \beta^2}  $$

## Using sympy to define stuff

In [2]:
import sympy
def create_area_symexp(
    s_alpha,
    s_beta,
    s_l_i,
):
    s_A = (sympy.sqrt(3)*3/2)*s_alpha*s_beta* (s_l_i**2)
    return s_A

def create_perim_symexp(
    s_alpha,
    s_beta,
    s_l_i,
):
    s_P = 2*s_alpha*s_l_i + 2*s_l_i*sympy.sqrt(s_alpha**2 + 3 * s_beta**2)
    return s_P

def area_energy_symexp(
    s_alpha,
    s_beta,
    s_l_i,
    s_Area_rest,
    s_k_A,
):
    s_Area = create_area_symexp(s_alpha, s_beta, s_l_i)
    return s_k_A * (s_Area - s_Area_rest)**2
    pass

def perimeter_energy_symexp(
    s_alpha,
    s_beta,
    s_l_i,
    s_P_r,
    s_k_P,
):
    s_Perimeter = create_perim_symexp(s_alpha, s_beta, s_l_i)
    return s_k_P * (s_Perimeter - s_P_r)**2
    

def create_energy_symexp(
    s_alpha,
    s_beta,
    s_l_i,
    s_Area_rest,
    s_Perimeter_rest,
    s_k_A,
    s_k_P,
):
    s_E_area = area_energy_symexp(s_alpha, s_beta, s_l_i, s_Area_rest, s_k_A)
    s_E_perim = perimeter_energy_symexp(s_alpha, s_beta, s_l_i, s_Perimeter_rest, s_k_P)
    
    
    s_E = s_E_area + s_E_perim
    
    return s_E
    

alpha, beta, l_i = sympy.symbols("alpha beta l_i", real=True, positive=True)
A_r, P_r = sympy.symbols("A_r P_r", real=True, positive=True)
k_A, k_P = sympy.symbols("k_A k_P", real=True)

energy_basic = create_energy_symexp(
    alpha,beta,l_i, A_r, P_r, k_A, k_P,
)

energy_basic

# A_r.is_real

k_A*(-A_r + 3*sqrt(3)*alpha*beta*l_i**2/2)**2 + k_P*(-P_r + 2*alpha*l_i + 2*l_i*sqrt(alpha**2 + 3*beta**2))**2

## What are the conditions for the hexagonal packing to be stable?
Maybe this is a wrong way to think about it? But I think that if the perimeter term wants the cell to be smaller than the area term does, then it will be stable (like a balloon). But if the area terms wants the cell to be smaller, then the stable situation will be a deflated cell - not good.

So assuming that $\alpha=\beta$, we want to find the stable $\alpha$ for the area term and for the perimeter term - depending on which one is bigger, that should tell us the stability of the system.

### Find the rest alpha for just area energy term

In [3]:
area_energy_isotropic = area_energy_symexp(
    alpha,
    beta,
    l_i,
    A_r,
    k_A
).subs([
    (beta,alpha),
    (k_A, 1),
])

sols_for_area_en_iso = sympy.solve(area_energy_isotropic.diff(alpha), alpha)

if len(sols_for_area_en_iso) != 1:
    print(sols_for_area_en_iso)
    raise Exception("Expected exactly one minimum for the area energy...")

alpha_rest_for_area = sols_for_area_en_iso[0]
# len(da_sol)

print("Alpha that minimizes the area energy term:")
alpha_rest_for_area

Alpha that minimizes the area energy term:


sqrt(2)*3**(1/4)*sqrt(A_r)/(3*l_i)

### Find the rest alpha for the perimeter term

In [4]:
perimeter_energy_isotropic = perimeter_energy_symexp(
    alpha,
    beta,
    l_i,
    P_r,
    k_P
).subs([
    (beta,alpha),
    (k_P, 1),
])

perimeter_energy_isotropic

sols_for_p_en_iso = sympy.solve(perimeter_energy_isotropic.diff(alpha), alpha)

if len(sols_for_p_en_iso) != 1:
    raise Exception("Expected exactly one energy minimum for the perimeter energy term...")
# sols_for_p_en_iso
alpha_rest_for_perimeter = sols_for_p_en_iso[0]

print("Alpha that minimizes the perimeter energy term:")
alpha_rest_for_perimeter

Alpha that minimizes the perimeter energy term:


P_r/(6*l_i)

## This is the inequality if we want to stay in the easy to deal with case:

$$ \alpha_{perim,rest} < \alpha_{area,rest} $$

In [4]:
# x_test = sympy.symbols("x")
# sympy.solveset((10000 / x_test) - 1 < 0, x_test, sympy.Reals)
print("The P_r 'rest perimeter' has to be less than this value:")

sympy.solveset(alpha_rest_for_perimeter < alpha_rest_for_area, P_r, sympy.Reals)

The P_r 'rest perimeter' has to be less than this value:


Interval.open(-oo, 2*sqrt(2)*3**(1/4)*sqrt(A_r))

## Now that we can assume $\alpha_{equilibrium} = \beta_{equilibrium}$, how do we find the equilibrium alpha for a given system?
Here is the energy, substituting $\beta = \alpha$ since our equilibrium cell should be stretched isotropically. 
$$\beta=\alpha$$

In [50]:
energy_expression_whittle_step_1 = energy_basic.subs([
    (beta, alpha),
])

sympy.Eq(
    sympy.Function('E')(alpha, beta, A_r, P_r, k_A, k_P, l_i),
    energy_expression_whittle_step_1,
)

Eq(E(alpha, beta, A_r, P_r, k_A, k_P, l_i), k_A*(-A_r + 3*sqrt(3)*alpha**2*l_i**2/2)**2 + k_P*(-P_r + 6*alpha*l_i)**2)

Any scaling of $l_i$ can be factored into $\alpha$, so we can say $\tilde{\alpha}=\alpha l_i$.

Also, the equilibrium condition $dE/d\alpha = 0$ shouldn't depend on the scaling of the entire energy, just the ratio of $k_A$ and $k_P$ - so we can also set $\tilde{k_P} = k_P/k_A$.
$$\tilde{\alpha}=\alpha l_i$$
$$\tilde{k_P} = k_P/k_A$$

In [59]:
alpha_til = sympy.symbols(r"\tilde{\alpha}")
k_P_til = sympy.symbols(r"\tilde{k_P}")
energy_expression_whittle_step_2 = energy_expression_whittle_step_1.subs([
    (alpha*l_i, alpha_til),
    (k_P, k_A * k_P_til),
])

sympy.Eq(
    sympy.Function('E')(alpha_til, A_r, P_r, k_A, k_P_til),
    
    energy_expression_whittle_step_2,
)

Eq(E(\tilde{\alpha}, A_r, P_r, k_A, \tilde{k_P}), \tilde{k_P}*k_A*(-P_r + 6*\tilde{\alpha})**2 + k_A*(-A_r + 3*sqrt(3)*\tilde{\alpha}**2/2)**2)

### Assuming $k_A > 0$, we can divide through by $k_A$ without changing the values of the energy minimums:

In [60]:
energy_expression_whittle_step_3 = (energy_expression_whittle_step_2 / k_A).simplify()

sympy.Eq(
    sympy.Function(r'\tilde{E}')(alpha_til, A_r, P_r, k_P_til),
    
    energy_expression_whittle_step_3,
)

Eq(\tilde{E}(\tilde{\alpha}, A_r, P_r, \tilde{k_P}), \tilde{k_P}*(P_r - 6*\tilde{\alpha})**2 + (2*A_r - 3*sqrt(3)*\tilde{\alpha}**2)**2/4)

The condition for equilibrium is that $d\tilde{E}/d\tilde{\alpha}$ = 0, so we take the derivative with respect to $\tilde{\alpha}$:

In [61]:
dE_dalpha_iso_exp = energy_expression_whittle_step_2.diff(alpha_til).expand()

print("Derivative of E with respect to alpha, holding other variables constant:")
sympy.Eq(
    (sympy.Function(r"\tilde{E}")(alpha_til, A_r, P_r, k_P_til)).diff(alpha_til),
    dE_dalpha_iso_exp,
)

Derivative of E with respect to alpha, holding other variables constant:


Eq(Derivative(\tilde{E}(\tilde{\alpha}, A_r, P_r, \tilde{k_P}), \tilde{\alpha}), -6*sqrt(3)*A_r*\tilde{\alpha}*k_A - 12*P_r*\tilde{k_P}*k_A + 27*\tilde{\alpha}**3*k_A + 72*\tilde{\alpha}*\tilde{k_P}*k_A)

It's kind of hard to find the roots in alpha when holding the $P_r$ and $\tilde{k_P}$ terms constant - so instead, we can find the $P_r$ that will make a given $\alpha$ a root, with also a given $k_P$:

In [62]:
# all_perim_rest_solutions_for_stable_at_given_alph = sympy.solve(dE_dalpha_iso_exp, P_r)

# if len(all_perim_rest_solutions_for_stable_at_given_alph) != 1:
#     raise Exception("Only expected one root...")

# all_perim_rest_solutions_for_stable_at_given_alph[0]

In [63]:
perim_rest_for_stable_alpha_TILDED = sympy.solve(dE_dalpha_iso_exp, P_r)[0]

perim_rest_for_stable_alpha_TILDED

\tilde{\alpha}*(-2*sqrt(3)*A_r + 9*\tilde{\alpha}**2 + 24*\tilde{k_P})/(4*\tilde{k_P})

### We implicitly have $\alpha_{stable}$ as a function of $P_r$, $A_r$, $k_P$, $k_A$, $l_i$

In [64]:
perim_rest_for_stable_alpha = perim_rest_for_stable_alpha_TILDED.subs([
    (alpha_til, alpha*l_i),
    (k_P_til, k_P/k_A),
])

sympy.Eq(
    P_r,
    perim_rest_for_stable_alpha.subs([
        (alpha, sympy.symbols(r"\alpha_{stable}")),
    ])
)

Eq(P_r, \alpha_{stable}*k_A*l_i*(-2*sqrt(3)*A_r + 9*\alpha_{stable}**2*l_i**2 + 24*k_P/k_A)/(4*k_P))

## Finding the poisson ratio

So, we can now (at least implicitly!) find the alpha that brings a cell to equilibrium.

## How do we find poisson ratio around the stable alpha?

In [65]:
alph_stable_dummy = sympy.symbols(r"\alpha_{stest}")

# alph_stable_dummy
energy_basic.diff(alpha).diff(beta).subs([
    (alpha, alph_stable_dummy),
    (beta, alph_stable_dummy),
])

27*\alpha_{stest}**2*k_A*l_i**4/2 - 3*\alpha_{stest}**2*k_P*l_i*(-P_r + 2*\alpha_{stest}*l_i + 4*l_i*sqrt(\alpha_{stest}**2))/(2*(\alpha_{stest}**2)**(3/2)) + 3*\alpha_{stest}*k_P*l_i*(2*\alpha_{stest}*l_i/sqrt(\alpha_{stest}**2) + 4*l_i)/sqrt(\alpha_{stest}**2) + 3*sqrt(3)*k_A*l_i**2*(-A_r + 3*sqrt(3)*\alpha_{stest}**2*l_i**2/2)

In [66]:

energy_basic.diff(beta).diff(beta).subs([
    (alpha, alph_stable_dummy),
    (beta, alph_stable_dummy),
])

27*\alpha_{stest}**2*k_A*l_i**4/2 - 9*\alpha_{stest}**2*k_P*l_i*(-P_r + 2*\alpha_{stest}*l_i + 4*l_i*sqrt(\alpha_{stest}**2))/(2*(\alpha_{stest}**2)**(3/2)) + 18*k_P*l_i**2 + 6*k_P*l_i*(-P_r + 2*\alpha_{stest}*l_i + 4*l_i*sqrt(\alpha_{stest}**2))/sqrt(\alpha_{stest}**2)

# TODO: Define alpha stable equals one, and work from there ?